## Introdução
Nosso grupo enfrentou algumas dificuldades na definição do projeto, equilibrando a busca por uma proposta relevante com o receio de estabelecer objetivos além de nossas capacidades. Após diversas discussões e ideias, decidimos desenvolver uma solução que compara dois textos e, por meio de análises semânticas e sintáticas, calcula a similaridade entre eles em forma de porcentagem. Dessa maneira, buscamos inferir se os textos foram ou não produzidos pelo mesmo autor. 
Após a definição da nossa ideia, iniciamos pesquisas sobre o plano de ação para lidar com o problema proposto. Primeiramente, percebemos que o pré-processamento é uma etapa extremamente relevante e que difere consideravelmente de outros casos de PLN. Isso ocorre porque um autor pode expressar seus maneirismos linguísticos e estilísticos de maneira sutil, o que torna características geralmente descartadas — como o início e o fim de sentenças, a pontuação e o uso de stopwords — potencialmente relevantes para a análise.

## Coleta de dados 
Considerando o problema abordado, optamos inicialmente por buscar textos de autores em comum, mas com temáticas distintas. Dessa forma, conseguimos testar a funcionalidade da nossa solução minimizando possíveis interferências decorrentes da similaridade temática entre os textos. Para isso, utilizamos o site Domínio Público, onde foi possível encontrar obras de diversos autores em língua portuguesa.
Selecionamos, nesta etapa, quatro textos: Dom Casmurro, de Machado de Assis, e O Cortiço, de Aluísio Azevedo e Iracema de José de Alencar. Atualmente, estamos utilizando esses 3 textos para testes, mas nossa intenção é expandir consideravelmente o conjunto de dados ao longo do desenvolvimento. Cada texto teve 4 capítulos extraídos e separados para análise.


In [71]:
import os

# Diretórios de origem e destino
origem = "data/caps"
destino = "data/caps_processados"
os.makedirs(destino, exist_ok=True)

for nome_arquivo in os.listdir(origem):
    caminho_origem = os.path.join(origem, nome_arquivo)
    caminho_destino = os.path.join(destino, os.path.splitext(nome_arquivo)[0] + ".txt")
    if os.path.isfile(caminho_origem):
        with open(caminho_origem, 'r', encoding='utf-8') as f_origem:
            conteudo = f_origem.read()
        with open(caminho_destino, 'w', encoding='utf-8') as f_destino:
            f_destino.write(conteudo)
        print(f"Arquivo salvo: {caminho_destino}")

Arquivo salvo: data/caps_processados\bras_cubas_machado_cap_1.txt
Arquivo salvo: data/caps_processados\bras_cubas_machado_cap_2.txt
Arquivo salvo: data/caps_processados\bras_cubas_machado_cap_3.txt
Arquivo salvo: data/caps_processados\bras_cubas_machado_cap_4.txt
Arquivo salvo: data/caps_processados\dom_casmurro_machado_cap_1.txt
Arquivo salvo: data/caps_processados\dom_casmurro_machado_cap_2.txt
Arquivo salvo: data/caps_processados\dom_casmurro_machado_cap_3.txt
Arquivo salvo: data/caps_processados\dom_casmurro_machado_cap_4.txt
Arquivo salvo: data/caps_processados\iracema_jose_de_alencar_cap_1.txt
Arquivo salvo: data/caps_processados\iracema_jose_de_alencar_cap_2.txt
Arquivo salvo: data/caps_processados\iracema_jose_de_alencar_cap_3.txt
Arquivo salvo: data/caps_processados\iracema_jose_de_alencar_cap_4.txt
Arquivo salvo: data/caps_processados\o_cortico_aluisio_azevedo_cap_1.txt
Arquivo salvo: data/caps_processados\o_cortico_aluisio_azevedo_cap_2.txt
Arquivo salvo: data/caps_processad

## Limpeza dos textos

Na etapa de pré-processamento definimos uma pipeline de limpeza de dados, essa é responsável por tornar os dados mais homogêneos, sem perder a complexidade. Além disso, aplicamos tokenização no texto, tomando cuidado para manter a divisão entre sentenças e manter pontuação e stopwords que podem ser relevantes para análise.
Abaixo definimos as configurações e funções usadas para limpeza do texto

A célula abaixo corrige um problma com a biblioteca nltk que estávamos tendo.

In [72]:
import nltk
from nltk.tokenize import word_tokenize

nltk.data.path.append('./.venv/nltk_data')  

nltk.download('punkt', download_dir='./.venv/nltk_data')
nltk.download('stopwords', download_dir='./.venv/nltk_data')


[nltk_data] Downloading package punkt to ./.venv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to ./.venv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
import re
import unicodedata
from nltk.tokenize import sent_tokenize

def limpeza_texto(texto):

    # 1. Corrigir quebras de linha excessivas
    texto = texto.replace('\n', '')
    
    # 2. Corrigir múltiplos espaços
    texto = re.sub(r'[ \t]+', ' ', texto)
    
    # 3. Remover pontuação desnecessária (mas manter . , ? — e aspas)
    texto = re.sub(r'[!;:()*[\]{}<>]', '', texto)
    
    # 4. Converter para minúsculas
    texto = texto.lower()
    
    # 5. Remover acentos
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))
    
    return texto

def limpar_e_tokenizar_texto(texto):
    # Limpa o texto primeiro
    texto_limpo = limpeza_texto(texto)
    
    sentences = sent_tokenize(texto_limpo, language='portuguese')
    # Remove sentenças vazias
    sentences = [s for s in sentences if s.strip()]
    # Tokeniza as sentenças e achata a lista de listas
    tokens = [token for s in sentences for token in word_tokenize(s, language='portuguese')]

    return tokens

def extrair_titulo_autor(nome_arquivo):
    # Remove "preprocessado_" e ".txt" ou ".json"
    nome_limpo = re.sub(r'^preprocessado_', '', nome_arquivo)
    nome_limpo = re.sub(r'\.(txt|json)$', '', nome_limpo)
    
    # Separar título e autor
    match = re.match(r'(.+?) \((.+?)\)', nome_limpo)
    if match:
        titulo, autor = match.groups()
        return titulo.strip(), autor.strip()
    else:
        return nome_limpo.strip(), None  # Se não bater, só retorna o título

### Tokenização e Armazenamento

Na célula abaixo realizamos a tokenização e armazenamento dos textos em um json com título do texto, nome do autor, e conteúdo tokenizado.

In [74]:
import os
import json

# Caminhos
caminho_txts = "data/caps"
caminho_saida_pasta = "data/caps_processados"
os.makedirs(caminho_saida_pasta, exist_ok=True)

for arquivo_nome in os.listdir(caminho_txts):
    dumped_dict = {}
    dumped_dict['titulo'], dumped_dict['autor'] = extrair_titulo_autor(arquivo_nome)
    dumped_dict['tokens'] = [] # Esta será uma lista de listas de tokens
    caminho_arquivo = os.path.join(caminho_txts, arquivo_nome)
    if os.path.isfile(caminho_arquivo):
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            texto_pre = arquivo.read()
            frases = texto_pre.split('.') # Divide o texto em segmentos baseados no ponto final

            for frase_segmento in frases: # Renomeei para clareza
                frase_limpa_para_tokenizar = frase_segmento.strip()
                if frase_limpa_para_tokenizar: # Pula segmentos vazios
                    print(f"\nProcessando frase original (após split e strip): '{frase_limpa_para_tokenizar}'") # DEBUG

                    # A função limpar_e_tokenizar_texto deve retornar uma lista plana de tokens
                    tokens_da_frase = limpar_e_tokenizar_texto(frase_limpa_para_tokenizar)

                    print(f"Tokens gerados: {tokens_da_frase}") # DEBUG
                    print(f"Tipo dos tokens gerados: {type(tokens_da_frase)}") # DEBUG
                    if tokens_da_frase and isinstance(tokens_da_frase[0], list):
                        print(f"ALERTA: Parece que os tokens estão aninhados desnecessariamente: {tokens_da_frase}") # DEBUG

                    # Adiciona a lista de tokens (que deve ser plana) à lista principal
                    dumped_dict['tokens'].append(tokens_da_frase)

            caminho_saida = os.path.join(caminho_saida_pasta, "preprocessado_" + arquivo_nome.replace('.txt', '.json'))
            with open(caminho_saida, 'w', encoding='utf-8') as arquivo_saida:
                json.dump(dumped_dict, arquivo_saida, ensure_ascii=False, indent=2)

            print(f"\nTokens estruturados salvos em: {caminho_saida}")


Processando frase original (após split e strip): 'Algum tempo hesitei se devia abrir estas memórias pelo princípio ou pelo fim, isto é, se poria em
primeiro lugar o meu nascimento ou a minha morte'
Tokens gerados: ['algum', 'tempo', 'hesitei', 'se', 'devia', 'abrir', 'estas', 'memorias', 'pelo', 'principio', 'ou', 'pelo', 'fim', ',', 'isto', 'e', ',', 'se', 'poria', 'emprimeiro', 'lugar', 'o', 'meu', 'nascimento', 'ou', 'a', 'minha', 'morte']
Tipo dos tokens gerados: <class 'list'>

Processando frase original (após split e strip): 'Suposto o uso vulgar seja começar pelo nascimento,
duas considerações me levaram a adotar diferente método: a primeira é que eu não sou propriamente
um autor defunto, mas um defunto autor, para quem a campa foi outro berço; a segunda é que o escrito
ficaria assim mais galante e mais novo'
Tokens gerados: ['suposto', 'o', 'uso', 'vulgar', 'seja', 'comecar', 'pelo', 'nascimento', ',', 'duas', 'consideracoes', 'me', 'levaram', 'a', 'adotar', 'diferente', 'meto

## Extração de features

Após a etapa de pré processamento, iniciaremos a extração de features. Definimos alguns grupos de features que acreditamos serem úteis para o nosso projeto, sendo elas: 
#### Linguísticas (POS, gramática, estilo):
- Frequência relativa de substantivos, verbos, adjetivos, advérbios
- Frequência de tempos verbais (passado, presente, futuro)
- Frequência de pronomes pessoais ("eu", "nós", etc.)
- Frequência de artigos definidos ("o", "a") e indefinidos ("um", "uma")
- Frequência de conjunções ("e", "mas", "porque")
- Número médio de palavras por frase
- Comprimento médio das palavras
- Variabilidade de comprimento de palavras (desvio padrão)
- Número de frases curtas vs. frases longas
- Uso de voz passiva (frases com "foi feito", "era conhecido", etc.)
- Proporção de substantivos abstratos vs. concretos (se você quiser ser avançado)

#### Lexicais (vocabulário):
- Número total de palavras (tokens)
- Número de palavras únicas (tipos)
- Índice de riqueza lexical: tipos / tokens
- Frequência de palavras raras (pouco frequentes em um corpus comum)
- Frequência de palavras comuns (ex: palavras do top-1000 do português)
- Uso de palavras sofisticadas (frequência de palavras acima de certo número de sílabas)

#### Semânticas (significado):
- Similaridade semântica média entre frases (usando embeddings como Word2Vec, BERT, etc.)
- Distância semântica entre parágrafos
- Frequência de negação ("não", "nunca", "jamais")
- Sentimento médio do texto (positivo/negativo/neutro)
- Frequência de emoções específicas (raiva, alegria, tristeza, surpresa)

#### Estilísticas:
- Uso de metáforas, hipérboles (difícil, mas dá pra tentar detectar por padrões)
- Uso de citações diretas ("...")
- Frequência de perguntas feitas ("?")
- Uso de primeira pessoa ("eu", "meu") vs. terceira pessoa ("ele", "ela")

#### Estatísticas avançadas:
- TF-IDF de palavras ou n-grams (unigrams, bigrams, trigrams)
- Topic Modeling (LDA) — para ver quais temas o autor tende a abordar
- Frequência de erros ortográficos (se tiver corpus sujo)
- Medidas de entropia do texto (quanto o texto é previsível)
Abaixo vamos buscar extrair features lexicais e semânticas dos textos. A partir disso, podemos comparar cada texto buscando entender melhor o que define os padrões na escrita de um autor.

In [75]:
import os
import json
from collections import Counter
import matplotlib.pyplot as plt
import re
import pandas as pd

# Caminho para os textos processados
caminho_textos_processados = "data/caps_processados"

def normalizar(valor, minimo, maximo, escala_min=1, escala_max=5):
    valor_normalizado = (valor - minimo) / (maximo - minimo)
    valor_normalizado = max(0, min(1, valor_normalizado)) 
    return escala_min + valor_normalizado * (escala_max - escala_min)

def diversidade_lexical(tokens):
    tokens_alfabeticos = [t.lower() for t in tokens if re.match(r'[a-zA-ZáàâãéèêíìóòôõúùûçÁÀÂÃÉÈÊÍÌÓÒÔÕÚÙÛÇ]+$', t)]
    num_tokens = len(tokens_alfabeticos)
    num_types = len(set(tokens_alfabeticos))

    # Inicializa todas as variáveis com valor padrão
    ttr = 0
    hapax_legomena = 0
    proporcao_hapax = 0
    score_ttr = 0
    score_hapax = 0
    score_diversidade = 1
    comprimento_medio_sentencas = 0
    comprimento_medio_palavras = 0
    media_stopwords = 0

    if num_tokens > 0:
        ttr = num_types / num_tokens
        contador = Counter(tokens_alfabeticos)
        hapax_legomena = sum(1 for palavra, freq in contador.items() if freq == 1)
        proporcao_hapax = hapax_legomena / num_tokens
        score_ttr = normalizar(ttr, minimo=0.0, maximo=1.0)
        score_hapax = normalizar(proporcao_hapax, minimo=0.0, maximo=1.0)
        score_diversidade = (score_ttr + score_hapax) / 2
        comprimento_medio_sentencas = sum(len(token) for token in tokens_alfabeticos) / num_tokens
        comprimento_medio_palavras = sum(len(token) for token in tokens_alfabeticos) / num_tokens
        stopwords = set(nltk.corpus.stopwords.words('portuguese'))
        media_stopwords = sum(1 for token in tokens_alfabeticos if token in stopwords) / num_tokens

    return {
        'ttr': ttr,
        'score_ttr': score_ttr,
        'comprimento_medio_sentencas': comprimento_medio_sentencas,
        'comprimento_medio_palavras': comprimento_medio_palavras,
        'media_stopwords': media_stopwords,
        'hapax_legomena': hapax_legomena,
        'proporcao_hapax': proporcao_hapax,
        'score_diversidade': score_diversidade,
        'score_palavras_unicas': score_hapax
    }

# Iterar sobre os textos processados e calcular a diversidade lexical
resultados_por_texto = []
for arquivo_nome in os.listdir(caminho_textos_processados):
    caminho_arquivo = os.path.join(caminho_textos_processados, arquivo_nome)
    if os.path.isfile(caminho_arquivo) and arquivo_nome.endswith('.json'):
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            dados = json.load(arquivo)
            tokens = [token for sublist in dados['tokens'] for token in sublist]
            resultado = diversidade_lexical(tokens)
            resultado['titulo'] = dados.get('titulo', 'Desconhecido')
            resultados_por_texto.append(resultado)

# Plotar os resultados
titulos = [r['titulo'] for r in resultados_por_texto]
scores_diversidade = [r['score_diversidade'] for r in resultados_por_texto]
ttrs = [r['ttr'] for r in resultados_por_texto]
proporcoes_hapax = [r['proporcao_hapax'] for r in resultados_por_texto]

# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados_por_texto)

# Reordenar as colunas para que o título seja o primeiro
colunas_reordenadas = ['titulo'] + [coluna for coluna in df_resultados.columns if coluna != 'titulo']
df_resultados = df_resultados[colunas_reordenadas]

# Salvar o DataFrame em um arquivo CSV
caminho_csv = os.path.join(caminho_textos_processados, 'resultados.csv')
df_resultados.to_csv(caminho_csv, index=False, encoding='utf-8')

In [76]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads

model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

def get_sentence_embeddings(sentences, model):
    """
    Obtém o embedding médio para cada sentença usando um modelo pré-treinado.

    Args:
        sentences (list): Lista de sentenças.
        model: Modelo de embedding.

    Returns:
        dict: Mapeamento de sentença para vetor de embedding.
    """
    embeddings = {}
    for i, sentence in enumerate(sentences):
        if not isinstance(sentence, str) or len(sentence) < 5:
            continue
        sentence_embedding = tokenizer.encode(sentence, return_tensors='pt')
        print(f"DEBUG: Tokenized sentence {i}: {sentence_embedding}")  # DEBUG
        embeddings[i] = sentence_embedding
    return embeddings

def analisar_coesao_sentences(sentences, words, embeddings):
    """
    Analisa a coesão e coerência do texto de forma detalhada.

    Args:
        sentences (list): Lista de frases do texto.
        words (list): Lista de palavras do texto.
        embeddings (dict): Embeddings das sentenças (mapeando texto -> vetor).

    Returns:
        dict: Resultados de análise de coesão, conectivos e similaridade semântica.
    """
    # --- Análise de conectivos ---
    conectivos = [
        'e', 'mas', 'porem', 'contudo', 'entretanto', 'portanto', 'assim', 'logo',
        'pois', 'porque', 'ja que', 'uma vez que', 'quando', 'enquanto', 'se', 'caso',
        'embora', 'apesar de', 'alem disso', 'ademais', 'ou seja', 'isto e', 'todavia',
        'nao obstante', 'ainda que', 'de modo que', 'de forma que', 'por conseguinte',
        'dessa forma', 'desse modo', 'conquanto', 'sobretudo', 'inclusive', 'nem',
        'tampouco', 'ou', 'ora', 'quer', 'seja', 'senao', 'assim como', 'bem como',
        'como', 'tal como', 'tanto quanto', 'quanto', 'do mesmo modo', 'igualmente',
        'em vez de', 'ao passo que', 'desde que', 'a fim de que', 'para que', 'antes que',
        'logo que', 'assim que', 'tao logo', 'depois que', 'porque', 'porquanto', 'visto que',
        'posto que', 'uma vez que', 'ja que', 'em virtude de', 'em razao de', 'gracas a',
        'apesar disso', 'mesmo assim', 'de qualquer forma', 'de qualquer maneira', 'por outro lado',
        'em contrapartida', 'em contraste', 'ao contrario', 'pelo contrario', 'no entanto',
        'de fato', 'efetivamente', 'realmente', 'com efeito', 'por exemplo', 'alias', 'a proposito',
        'inclusive', 'alem do mais', 'acima de tudo', 'principalmente', 'sobretudo', 'nao so',
        'como tambem', 'bem como', 'nao apenas', 'mas tambem', 'tanto', 'quanto', 'se nao', 'caso contrario',
        'a medida que', 'a proporcao que', 'de maneira que', 'de sorte que', 'de tal forma que',
        'de tal modo que', 'de forma que', 'de jeito que', 'de maneira que', 'de modo que',
        'em funcao de', 'em vista de', 'por causa de', 'por conta de', 'por motivo de', 'por razao de',
        'em consequencia', 'em decorrencia', 'em resultado', 'em virtude', 'em vista disso', 'por isso',
        'por essa razao', 'por esse motivo', 'por essa causa', 'por essa razao', 'por esse motivo'
    ]
    conectivos_encontrados = []
    for c in conectivos:
        for sentence in sentences:
            sentence = sentence.lower()  # Normaliza para minúsculas
            if c in sentence:
                conectivos_encontrados.append(c)
                break
    num_conectivos = len(conectivos_encontrados)
    num_tokens = len(words)
    proporcao_conectivos = num_conectivos / (num_tokens + 1e-6)  # Evitar divisão por zero

    # --- Análise de coesão semântica ---
    # Garante que todos os vetores de embeddings sejam válidos
    vetores = []
    for vetor in embeddings.values():
        if vetor is not None and len(vetor) > 0:
            vetor = vetor.T
            vetores.append(vetor)
    if not vetores:
        print("Nenhum vetor de embedding válido encontrado. Retornando coesão padrão.")
        return {
            'coesao_score': 0.0,
            'conectivos_encontrados': conectivos_encontrados,
            'num_conectivos': num_conectivos,
            'proporcao_conectivos': round(proporcao_conectivos, 3),
            'similaridade_media': 0.0,
            'num_sentencas': len(sentences)
        }
    
    if len(vetores) < 2:
        similaridade_media = 1.0  # Texto curto, assume alta coesão
        print("isso aqui rodou.\n") # DEBUG
    else:
        similaridades = []
        for i in range(len(vetores) - 1):
            try:
                sim = cosine_similarity(vetores[i], vetores[i+1])[0][0]
                similaridades.append(sim)
            except Exception as e:
                print(f"Vetores possuem os tamanhos diferentes: {len(vetores[i])} e {len(vetores[i+1])}. Erro: {e}")
        similaridade_media = np.mean(similaridades)

    # --- Cálculo do Score Final ---
    # Peso 50% conectivos, 50% semântica
    score_conectivos = min(1, proporcao_conectivos)  # Ideal: 10% de conectivos
    score_semantica = similaridade_media      # Garante entre 0 e 1

    coesao_score_final = (score_conectivos + score_semantica) / 2
    print(f"DEBUG ======================\n len vetores {len(vetores)} \nSimilaridade média: {similaridade_media}\n Proporção de conectivos: {proporcao_conectivos}\n Coesão Score Final: {coesao_score_final}\n Conectivos encontrados: {conectivos_encontrados}\n Número de conectivos: {num_conectivos}\n Número de sentenças: {len(sentences)}\n======================")  # DEBUG
    return {
        'coesao_score': round(coesao_score_final, 2),
        'conectivos_encontrados': conectivos_encontrados,
        'num_conectivos': num_conectivos,
        'proporcao_conectivos': round(proporcao_conectivos, 3),
        'similaridade_media': round(similaridade_media, 3),
        'num_sentencas': len(sentences)
    }

# Iterar sobre os textos processados
caminho_saida_pasta = "data/caps_processados"
for arquivo_nome in os.listdir(caminho_saida_pasta):
    caminho_arquivo = os.path.join(caminho_saida_pasta, arquivo_nome)
    if os.path.isfile(caminho_arquivo) and arquivo_nome.endswith('.json'):
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            dados = json.load(arquivo)
            sentences = [" ".join(tokens) for tokens in dados['tokens']]
            tokens = [token for sublist in dados['tokens'] for token in sublist]
            titulo = dados.get('titulo', 'Desconhecido')
            # Obter embeddings e analisar coesão
            embeddings = get_sentence_embeddings(sentences, model)
            resultados = analisar_coesao_sentences(sentences=sentences, words=tokens, embeddings=embeddings)
            
            print(f"Resultados para {arquivo_nome}:")
            print(resultados)
            print("\n")
            import matplotlib.pyplot as plt

            # Criando um gráfico consolidado para todas as métricas por livro
            plt.figure(figsize=(12, 8))

            # Adiciona os resultados do livro atual à lista
            # Verifica se o texto já está presente no DataFrame
            if titulo in df_resultados['titulo'].values:
                # Atualiza a linha correspondente com os novos resultados
                df_resultados.loc[df_resultados['titulo'] == titulo, ['proporcao_conectivos', 'similaridade_media', 'coesao_score', 'num_conectivos', 'num_sentencas']] = [
                    resultados['proporcao_conectivos'],
                    resultados['similaridade_media'],
                    resultados['coesao_score'],
                    resultados['num_conectivos'],
                    resultados['num_sentencas']
                ]
            else:
                # Adiciona uma nova linha com os resultados, incluindo as colunas ausentes
                df_resultados = pd.concat([
                    df_resultados,
                    pd.DataFrame([{
                        'titulo': titulo,
                        'proporcao_conectivos': resultados['proporcao_conectivos'],
                        'similaridade_media': resultados['similaridade_media'],
                        'coesao_score': resultados['coesao_score'],
                        'ttr': None,
                        'score_ttr': None,
                        'num_types': None,
                        'num_tokens': None,
                        'proporcao_hapax': None,
                        'score_diversidade': None,
                        'score_palavras_unicas': None
                    }])
                ], ignore_index=True)

# Salva os resultados atualizados em um arquivo CSV
caminho_csv = os.path.join(caminho_saida_pasta, 'resultados.csv')
df_resultados.to_csv(caminho_csv, index=False, encoding='utf-8')

DEBUG: Tokenized sentence 0: tensor([[  101,  3179,   596, 18540,   593, 22283,   176, 12444,  8925,  3769,
         14876,   562,   423,   905,   247,   291,   423,  1338,   117,  3413,
           122,   117,   176,   240,   151,  4276,  3198,   458,  1247,   146,
          7343,  5774,   291,   123,  7122,  1386,   102]])
DEBUG: Tokenized sentence 1: tensor([[  101, 15581,   146,  1700, 21928,  1547,   662,   934,   423,  5774,
           117,   924,   996,   303,   143,   311,  6777,   123, 14396,  3575,
         11637, 22280,   123,   681,   122,   179,  2779,   229, 22280,  7206,
         12762,   389,  1368,   975,  5242, 22280,   117,   449,   222,   975,
          5242, 22280,  1368,   117,   221,  1977,   123,  5036, 22278,   262,
          1342, 10420,   303,   123,  1448,   122,   179,   146,  2685,  1282,
           322,  1016,   325, 22231,   175,   122,   325,  1160,   102]])
DEBUG: Tokenized sentence 2: tensor([[  101,   390,  7360,   117,   179, 14619,   210,  5222,   1

C:\Users\turevs\AppData\Local\Temp\ipykernel_8244\167791990.py:144: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 8))


DEBUG: Tokenized sentence 78: tensor([[  101,   527,  3004,   151,  1636,   203,   118,   176,   123,  9317,
           125,   495,  1087,   117, 16635,   214,   146,  5023,   428,   123,
          9166,   123,   661,  5843, 22278,   179,  2036,  9086,   975,  1885,
           185,   102]])
DEBUG ======================
 len vetores 76 
Similaridade média: 1.0
 Proporção de conectivos: 0.016983695640636075
 Coesão Score Final: 0.5084918478203181
 Conectivos encontrados: ['e', 'mas', 'assim', 'logo', 'pois', 'quando', 'se', 'caso', 'apesar de', 'sobretudo', 'nem', 'ou', 'ora', 'quer', 'seja', 'senao', 'como', 'quanto', 'em vez de', 'para que', 'ao contrario', 'sobretudo', 'nao so', 'quanto', 'se nao']
 Número de conectivos: 25
 Número de sentenças: 79
Resultados para preprocessado_senhora_jose_de_alencar_cap_3.json:
{'coesao_score': np.float64(0.51), 'conectivos_encontrados': ['e', 'mas', 'assim', 'logo', 'pois', 'quando', 'se', 'caso', 'apesar de', 'sobretudo', 'nem', 'ou', 'ora', 'quer

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

...

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [77]:
import spacy
from collections import Counter
import os
import json
import pandas as pd

caminho_textos_processados = "data/caps_processados" 

nlp = spacy.load("pt_core_news_sm")

arquivos_json = [f for f in os.listdir(caminho_textos_processados) if f.endswith('.json')]

def contar_pos_tags(texto):
    doc = nlp(texto)
    contagem = Counter()
    for token in doc:
        if not token.is_punct and not token.is_space:
            contagem[token.pos_] += 1
    total_tokens = sum(contagem.values())
    if total_tokens > 0:
        proporcoes = {pos: qtd / total_tokens for pos, qtd in contagem.items()}
    else:
        proporcoes = {pos: 0 for pos in contagem}
    return proporcoes

contagens_por_arquivo = {}
contagem_total = Counter()

for nome_arquivo in arquivos_json:
    caminho_completo = os.path.join(caminho_textos_processados, nome_arquivo)
    with open(caminho_completo, 'r', encoding='utf-8') as f:
        dados = json.load(f)
        texto = " ".join([token for sublist in dados['tokens'] for token in sublist])
        contagem = contar_pos_tags(texto)
        contagens_por_arquivo[nome_arquivo] = contagem
        contagem_total.update(contagem)

# Carregar o DataFrame existente
caminho_csv = os.path.join(caminho_textos_processados, 'resultados.csv')
df = pd.read_csv(caminho_csv)

# Para cada arquivo, adicionar/atualizar as features POS no DataFrame
for nome_arquivo, contagem in contagens_por_arquivo.items():
    # Remover prefixo "preprocessado_" e sufixo ".json" para bater com o campo 'titulo'
    titulo = nome_arquivo.replace("preprocessado_", "").replace(".json", "")
    for pos, qtd in contagem.items():
        # Adiciona a coluna se não existir
        if pos == "SYM" or pos == "PUNCT" or pos == "INTJ":
            continue
        if pos not in df.columns:
            df[pos] = 0.0
        # Atualiza o valor na linha correspondente ao título
        df.loc[df['titulo'] == titulo, pos] = qtd

# Salvar o DataFrame atualizado
df.to_csv(caminho_csv, index=False, encoding='utf-8')

print("\n Contagem por arquivo:")
for nome_arquivo, contagem in contagens_por_arquivo.items():
    print(f"\nArquivo: {nome_arquivo}")
    for pos, qtd in contagem.items():
        print(f"  {pos}: {qtd}")


 Contagem por arquivo:

Arquivo: preprocessado_bras_cubas_machado_cap_1.json
  DET: 0.14573643410852713
  NOUN: 0.2682170542635659
  VERB: 0.14263565891472868
  PRON: 0.07751937984496124
  ADP: 0.11162790697674418
  CCONJ: 0.06666666666666667
  ADJ: 0.05426356589147287
  AUX: 0.021705426356589147
  NUM: 0.018604651162790697
  SCONJ: 0.03875968992248062
  ADV: 0.05116279069767442
  PROPN: 0.0015503875968992248
  X: 0.0015503875968992248

Arquivo: preprocessado_bras_cubas_machado_cap_2.json
  ADP: 0.1362126245847176
  NOUN: 0.28903654485049834
  NUM: 0.009966777408637873
  VERB: 0.15614617940199335
  SCONJ: 0.05647840531561462
  DET: 0.14950166112956811
  PRON: 0.06312292358803986
  AUX: 0.013289036544850499
  ADV: 0.046511627906976744
  CCONJ: 0.019933554817275746
  ADJ: 0.046511627906976744
  PROPN: 0.009966777408637873
  X: 0.0033222591362126247

Arquivo: preprocessado_bras_cubas_machado_cap_3.json
  CCONJ: 0.050473186119873815
  PROPN: 0.015772870662460567
  PRON: 0.0630914826498422

In [78]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Carregar o DataFrame
df = pd.read_csv('data/caps_processados/resultados.csv')

# Selecionar apenas as colunas numéricas (ignorando 'titulo')
X = df.drop(columns=['titulo'])

# Padronizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definir o número de clusters (exemplo: 3, ajuste conforme necessário)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Adicionar o resultado ao DataFrame
df['cluster'] = clusters

# Exibir os clusters
print(df[['titulo', 'cluster']])

                             titulo  cluster
0          bras_cubas_machado_cap_1        2
1          bras_cubas_machado_cap_2        2
2          bras_cubas_machado_cap_3        2
3          bras_cubas_machado_cap_4        2
4        dom_casmurro_machado_cap_1        0
5        dom_casmurro_machado_cap_2        2
6        dom_casmurro_machado_cap_3        0
7        dom_casmurro_machado_cap_4        2
8     iracema_jose_de_alencar_cap_1        2
9     iracema_jose_de_alencar_cap_2        2
10    iracema_jose_de_alencar_cap_3        2
11    iracema_jose_de_alencar_cap_4        2
12  o_cortico_aluisio_azevedo_cap_1        1
13  o_cortico_aluisio_azevedo_cap_2        1
14  o_cortico_aluisio_azevedo_cap_3        1
15  o_cortico_aluisio_azevedo_cap_4        1
16   o_mulato_aluisio_azevedo_cap_1        1
17   o_mulato_aluisio_azevedo_cap_2        1
18   o_mulato_aluisio_azevedo_cap_3        1
19   o_mulato_aluisio_azevedo_cap_4        1
20    senhora_jose_de_alencar_cap_1        2
21    senh

### Conclusão
Primeiramente, um fator essencial que notamos é que teremos que tratar a sensibilidade que existe à diferença de tamanho entre os textos, essa é uma questão na qual vamos precisar pensar com cuidado, evitar usar números absolutos será relevante para tornas os parâmetros mais comparáveis. Por fim, nessa próxima etapa nossa intenção é de aumentar a quantidade de features e principalmente o escopo dos dados, utilizar mais autores e textos para nossa análise será essencial mais para frente.